In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder \
    .appName('DataFrame') \
    .master('local[*]') \
    .getOrCreate()

In [44]:
def users_to_json(spark_context, file):
    def json_lambda(line):
        json_line = {}
        tags_list={'Id':int,
                   'Reputation':int,}
#                    'CreationDate':lambda x: str(x)[1:-3],
#                    'DisplayName':lambda x: str(x)[1:-3],
#                    'LastAccessDate':lambda x: str(x)[1:-3],
#                    'WebsiteUrl':lambda x: str(x)[1:-3],
#                    'Location':lambda x: str(x)[1:-3],
#                    'AboutMe':str,
#                    'Views':int,
#                    'UpVotes':int,
#                    'DownVotes':int,
#                    'EmailHash':lambda x: str(x)[1:-3],
#                    'AccountId':int,
#                    'Age':float}
        indices = [line.find(tag,0) for tag in tags_list if line.find(tag,0)!=-1]
        for i in range(len(indices)-1):
            parsed = line[indices[i]:indices[i+1]].split('=')
            try:
                json_line[parsed[0]] = tags_list[parsed[0]](parsed[1])
            except ValueError:
                json_line[parsed[0]] = tags_list[parsed[0]](parsed[1][1:-2])
        all_keys_json = {}
        for tag in tags_list:
            try:
                all_keys_json[tag] = json_line[tag]
            except KeyError:
                all_keys_json[tag] = None
        return all_keys_json
    usersRDD = spark_context.textFile(file)
    headers = usersRDD.take(2)
    usersRDD = usersRDD.filter(lambda x: x not in  headers)
    usersRDD = usersRDD.map(json_lambda)
    return usersRDD



In [45]:
import re
data_path = '/home/piotr/big_data/archive.org/download/stackexchange/'
import os
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

subdirs = get_immediate_subdirectories(data_path)
for subdir in subdirs[:1]:
    if not re.search('meta', subdir):
        name = subdir.split('.')[0]
        try:
            RDD_json = users_to_json(spark.sparkContext, data_path + subdir + '/Users.xml')
            RDD_json.saveAsTextFile('./jsons/' + name)
        except:
            print(name)

In [55]:
RDD_json.map(str).saveAsTextFile('./jsons/' + 'dsad')

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 33, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/piotr/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/home/piotr/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/piotr/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/piotr/anaconda3/lib/python3.7/site-packages/pyspark/rdd.py", line 1354, in takeUpToNumLeft
   